<a href="https://colab.research.google.com/github/sam4coding/pytorch_connectomics/blob/master/Colab_MitoEM_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install the required packages: ###

Testing using !pip install git+url instead of copying the whole git page to local

In [1]:
!pip install -q torch torchvision

In [2]:
# Install PyTorch Connectomics
!git clone https://github.com/zudi-lin/pytorch_connectomics.git
# In colab, we use the magic command '%cd', instead of '!cd' to navigate in the file systems
%cd pytorch_connectomics
!pip install --editable .

Cloning into 'pytorch_connectomics'...
remote: Enumerating objects: 11576, done.
remote: Counting objects: 100% (2842/2842), done.
remote: Compressing objects: 100% (1126/1126), done.
remote: Total 11576 (delta 1771), reused 2622 (delta 1659), pack-reused 8734
Receiving objects: 100% (11576/11576), 93.53 MiB | 28.06 MiB/s, done.
Resolving deltas: 100% (7450/7450), done.
/content/pytorch_connectomics
Obtaining file:///content/pytorch_connectomics
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.1 

### Setup for local (Google drive) saving path ###



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# set up an output folder
import os
output_folder = '/content/gdrive/MyDrive/mitoEM/output'
os.makedirs(output_folder, exist_ok=True)

### Download the pre-trained MitoEM model: ###

model: https://huggingface.co/pytc/mito/blob/main/mito_u3d-bc_mitoem_300k.pth.tar

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from huggingface_hub import hf_hub_download

filename = "mito_u3d-bc_mitoem_300k.pth.tar"
repo_id = "pytc/mito"
revision = "main"

checkpoint_path = hf_hub_download(repo_id=repo_id, filename=filename, revision=revision)
checkpoint_path

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mito_u3d-bc_mitoem_300k.pth.tar:   0%|          | 0.00/29.3M [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--pytc--mito/snapshots/4e9cdbaf05cac9b450947760f973d58b0c517d36/mito_u3d-bc_mitoem_300k.pth.tar'

In [7]:
# check the file size to make sure the import is done correctly.

import os

# Get file size in bytes
file_size_bytes = os.path.getsize(checkpoint_path)

# Convert to megabytes
file_size_megabytes = file_size_bytes / (1000 * 1000)

file_size_megabytes

# print(f"The file size is {file_size_megabytes} MB")
# print('a')

29.336963

### Run the inference ###

In [8]:
# get the test image
from huggingface_hub import hf_hub_download

from connectomics.data.utils import readh5

test_filename = "mitoem_R_train_4um_im.h5"
test_repo_id = "pytc/MitoEM"
test_revision = "main"

test_file_path = hf_hub_download(repo_id=test_repo_id, filename=test_filename, revision=test_revision, repo_type='dataset')
test_img = readh5(test_file_path)
# test_img = readh5("https://huggingface.co/datasets/pytc/MitoEM/blob/main/mitoem_R_train_4um_im.h5")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

mitoem_R_train_4um_im.h5:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

In [9]:
#Load the ground truth
from huggingface_hub import hf_hub_download

from connectomics.data.utils import readh5

gt_filename = "mitoem_R_train_4um_seg.h5"
gt_repo_id = "pytc/MitoEM"
gt_revision = "main"

gt_file_path = hf_hub_download(repo_id=gt_repo_id, filename=gt_filename, revision=gt_revision, repo_type='dataset')
gt_img = readh5(gt_file_path)

mitoem_R_train_4um_seg.h5:   0%|          | 0.00/536k [00:00<?, ?B/s]

In [10]:
# !CUDA_VISIBLE_DEVICES=0 python -u pytorch_connectomics/scripts/main.py
!python -u scripts/main.py \
 --config-file /content/pytorch_connectomics/configs/Lucchi-Mitochondria.yaml --inference \
 --checkpoint {checkpoint_path} \
 INFERENCE.IMAGE_NAME {test_file_path} \
 INFERENCE.OUTPUT_PATH {output_folder}


2024-05-13 16:40:32.157468: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 16:40:32.157520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 16:40:32.158806: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 16:40:33.262553: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/pytorch_connectomics/connectomics/config/utils.py:115: UserWarning: When downsampling by stride instead of using pooling layers, the cfg.MODEL.INPUT_SIZE are expected to contain numbers of 2n+1 to avoid feature mis-matching, but get

In [11]:
from connectomics.utils.evaluate import get_binary_jaccard
from connectomics.data.utils import readh5
import numpy as np

pred=readh5('/content/gdrive/MyDrive/mitoEM/output/result_txyz.h5')
pred = pred / 255.  # output is casted to uint8 with range [0,255].
gt = (gt_img != 0).astype(np.uint8)
thres = [0.4, 0.6, 0.8]  # evaluate at multiple thresholds.
scores = get_binary_jaccard(pred, gt, thres)
print(scores)

[[0.02217566 0.13324873 0.02412817 0.21509312]
 [0.00095378 0.28130672 0.00115983 0.00534007]
 [0.00115542 0.46115569 0.13294521 0.00116419]]
